In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

from sklearn.preprocessing import LabelEncoder

import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score,confusion_matrix,classification_report
pip install xgboost

In [ ]:
#importing the dataset
df_churn=pd.read_csv("data/Churn_Data.csv")
df_churn.info()
df_churn.isnull().sum()
df_churn.head()
df_churn.drop(['Surname'],axis=1,inplace=True)
df_churn.drop(['Gender'],axis=1,inplace=True)
df_churn.drop(['RowNumber'],axis=1,inplace=True)
df_churn.drop(['CustomerId'],axis=1,inplace=True)

In [ ]:
palette={"Germany": "#F0E100", "France": "#01295F", "Spain": "#F17105"}
palette_df=pd.DataFrame(palette.items(), columns=['Geography', 'Color'])
palette_df
customersa=pd.DataFrame(df_churn['Geography'].value_counts(normalize=False))
customersa=customersa.reset_index().rename(columns = {'index':'Geography','Geography':'Count'})

customers=pd.merge(customersa, palette_df, on=['Geography'],how='outer')

fig = go.Figure(data=[go.Pie(labels=customers['Geography'], values=customers['Count'],textinfo='label',
                     name='Customers Geographical Locations', 
                     marker_colors=customers['Color'],hole=0.5)])
fig.show()
#Churners and Non-Churners based on their country
churners=df_churn[df_churn['Exited']==1]
nonchurners=df_churn[df_churn['Exited']==0]

churners_counta=pd.DataFrame(churners['Geography'].value_counts(normalize=False))
churners_counta=churners_counta.reset_index().rename(columns = {'index':'Geography','Geography':'Count'})
churners_count=pd.merge(churners_counta, palette_df, on=['Geography'],how='outer')


nonchurners_counta=pd.DataFrame(nonchurners['Geography'].value_counts(normalize=False))
nonchurners_counta=nonchurners_counta.reset_index().rename(columns = {'index':'Geography','Geography':'Count'})
nonchurners_count=pd.merge(nonchurners_counta, palette_df, on=['Geography'],how='outer')

fig = make_subplots(rows=1, cols=2, 
                    specs=[[{"type": "pie"},{"type": "pie"}]],
                    subplot_titles=("Churners by Location","Non-churners by Location"))

fig.add_trace(go.Pie(labels=churners_count['Geography'], values=churners_count['Count'],textinfo='label',
                     name='Churners Geographical Locations', 
                     marker_colors=churners_count['Color'],hole=0.5),1,1)

fig.add_trace(go.Pie(labels=nonchurners_count['Geography'], values=nonchurners_count['Count'],textinfo='label',
                     name='Non-Churners Geographical Locations',  
                     marker_colors=nonchurners_count['Color'],hole=0.5),1,2)

In [ ]:
#Random Forest
sc = StandardScaler()
X_rftrain = sc.fit_transform(X_train)
X_rftest = sc.transform(X_test)
rfc = RandomForestClassifier(criterion ='gini', n_estimators=100,random_state = 10,max_features='auto')
rfc.fit(X_rftrain, y_train)


rfc_pred_train = rfc.predict(X_rftrain)
rfc_pred_test = rfc.predict(X_rftest)

confusion_matrix(y_test, rfc_pred_test)

accuracy_rf=accuracy_score(y_test, rfc_pred_test)*100
print("The accuracy score of the Random Forest Classification Model is: ",accuracy_rf)
print(classification_report(y_test, rfc_pred_test))